# FaultTree

## Install

Use Pkg to install the packages. The packages are located in GitHub. Please run the following commands:

- For Julia 1.0.5
```julia
using Pkg
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/DD.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/FaultTree.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/JuliaDot.jl.git"))
```

In [ ]:
using Pkg
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/SymbolicDiff.jl"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/DD.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/FaultTree.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/JuliaDot.jl.git"))

## Initialize

Load `FaultTree` and `JuliaDot`. `JuliaDot` provides drawing a graph of fault trees and BDD, but the package `JuliaDot` depends on your own environment. If the loding `JuliaDot` fails, please remove (comment out) loading `JuliaDot`

In [1]:
using FaultTree
using JuliaDot

### Example 1

#### Definition of FT model

In [58]:
@ftree ft(ra, rb, rc) begin
    @basic begin # define basic events
        x = ra
        y = rb
        z = rc
    end
    system = x & y & z
end

ft (generic function with 2 methods)

#### instantinate a FT model

In [59]:
system = ft(0.9999, 0.9999, 0.9999) # assign reliability probabilities as 0.9999

"node6"

#### Minimal path (cut) set

In [60]:
mcs(system)

1-element Vector{Vector{FaultTree.AbstractFTEvent}}:
 [:x, :y, :z]

#### Eval top event (system reliability)

In [48]:
prob(system)

0.9997000299990001

### Example 2

In [49]:
using SymbolicDiff

#### Definition of FT model

In [61]:
@ftree ft(ra, rb, rc) begin
    @basic begin # define basic events
        x = ra
        y = rb
        z = rc
    end
    system = x & y & z
end

ft (generic function with 2 methods)

#### instantinate a FT model with symbolic parameters

In [81]:
@vars begin
    t
    lama
    lamb
    lamc
end
system = ft(exp(-lama*t), exp(-lamb*t), exp(-lamc*t))

"node6"

#### Set parameters

In [97]:
env = SymbolicEnv()
env[lama] = 2.0833e-6
env[lamb] = 1.8182e-6
env[lamc] = 1.4285e-6
env[t] = 50.0

50.0

#### Eval top event

In [98]:
seval(prob(system), env)

0.9997335355079707

#### Birunbaum importance measure (the first derivative of top event prob with respect to failure probability)

In [99]:
seval(prob(system), :lama, env) / seval(exp(-lama*t), :lama, env)

0.9998376781756133

In [100]:
seval(prob(system), :lamb, env) / seval(exp(-lamb*t), :lamb, env)

0.9998244254150219

In [101]:
seval(prob(system), :lamc, env) / seval(exp(-lamc*t), :lamc, env)

0.9998049440258907

#### Birunbaum importance measure

In [107]:
@vars begin
    zero
    one
end

env[zero] = 0.0
env[one] = 1.0

b1 = prob(ft(one, exp(-lamb*t), exp(-lamc*t))) - prob(ft(zero, exp(-lamb*t), exp(-lamc*t)));
b2 = prob(ft(exp(-lama*t), one, exp(-lamc*t))) - prob(ft(exp(-lama*t), zero, exp(-lamc*t)));
b3 = prob(ft(exp(-lama*t), exp(-lamb*t), one)) - prob(ft(exp(-lama*t), exp(-lamb*t), zero));

# criticality
c1 = b1 * exp(-lama*t) / prob(system);
c2 = b2 * exp(-lamb*t) / prob(system);
c3 = b3 * exp(-lamc*t) / prob(system);

In [103]:
seval(b1, env)

0.9998376781756132

In [104]:
seval(b2, env)

0.9998244254150218

In [105]:
seval(b3, env)

0.9998049440258907

#### Epstemic uncertainty propagation

In [130]:
using Random
using Distributions

In [162]:
varcov = zeros(3,3)
n = 2
for (i,lam) = enumerate([lama, lamb, lamc])
    x = rand(Exponential(1/seval(lam, env)), n)
    e, s = mean(Gamma(n, 1/sum(x))), var(Gamma(n, 1/sum(x)))
    varcov[i,i] = s
end
varcov

3×3 Matrix{Float64}:
 1.35945e-11  0.0          0.0
 0.0          1.90064e-13  0.0
 0.0          0.0          3.87879e-13

In [163]:
exv = []
sv = []
for b = [b1, b2, b3]
    v1 = [seval(b, x, env) for x = [:lama, :lamb, :lamc]] # first derivative of BI_1
    v2 = [seval(b, (x,y), env) for x = [:lama, :lamb, :lamc], y=[:lama, :lamb, :lamc]] # second derivative of BI_1
    ex = seval(b, env) + sum(varcov .* v2)/2
    ex2 = seval(b, env)^2 + sum([(v1[i]^2 + seval(b,env) * v2[i,i]) * varcov[i,i] for (i,x) = enumerate([:lama, :lamb, :lamc])])
    push!(exv, ex)
    push!(sv, ex2 - ex^2)
end

In [164]:
exv

3-element Vector{Any}:
 0.9998376788979247
 0.9998244428899434
 0.9998049612532516

In [165]:
sv

3-element Vector{Any}:
 1.44438849769557e-9
 3.494370648393641e-8
 3.444800067420317e-8

In [110]:
seval(c3, env)

1.0

In [78]:
seval(c1, :lama, env)

-1.422601156349325e-14

### Example 2

#### Definition of component failure events

In [ ]:
# x = [ftevent("motor failure ", i) for i = ["arm1", "arm2", "arm3"]]
# y = [ftevent("sensor failure ", i) for i = ["left", "right"]]
# z = ftevent("battery failure")
@ftree ft(motor_failure, sensor_failure, battery_failure) begin
    @basic begin # define basic events
        for i = 1:3
            
        y = sensor_failure
        z = battery_failure
    end
    system = x | y | z
end

#### Definition of fault tree to represent the system failure

In [ ]:
sub = [x[i] + z for i = 1:3]
system = ftkofn(2, sub[1], sub[2], sub[3]) + (y[1] * y[2])

#### Drawing a brief fault tree.

In [ ]:
## println(todot(system))
draw(todot(system))

In [ ]:
top, forest = bdd(system)
draw(todot(forest, top))

#### Minimal cut set

In [ ]:
ftmcs(system)

#### Define failure probabilities for compoments

In [ ]:
params = Dict(
    [x[i].var => 0.001 for i = 1:3]...,
    [y[i].var => 0.0001 for i = 1:2]...,
    z.var => 0.00001
)

#### Compute the system failure probability

In [ ]:
fteval(system, params)

#### Birunbaum importance measure

In [ ]:
# define the derivative for a parameter.
dparams_x1 = Dict(x[1].var => 1.0, x[2].var => 0.0, x[3].var => 0.0, y[1].var => 0.0, y[2].var => 0.0, z.var => 0.0)
dparams_x2 = Dict(x[1].var => 0.0, x[2].var => 1.0, x[3].var => 0.0, y[1].var => 0.0, y[2].var => 0.0, z.var => 0.0)
dparams_x3 = Dict(x[1].var => 0.0, x[2].var => 0.0, x[3].var => 1.0, y[1].var => 0.0, y[2].var => 0.0, z.var => 0.0)
dparams_y1 = Dict(x[1].var => 0.0, x[2].var => 0.0, x[3].var => 0.0, y[1].var => 1.0, y[2].var => 0.0, z.var => 0.0)
dparams_y2 = Dict(x[1].var => 0.0, x[2].var => 0.0, x[3].var => 0.0, y[1].var => 0.0, y[2].var => 1.0, z.var => 0.0)
dparams_z = Dict(x[1].var => 0.0, x[2].var => 0.0, x[3].var => 0.0, y[1].var => 0.0, y[2].var => 0.0, z.var => 1.0)

In [ ]:
# importance for x
using Printf
@printf("%20s %.5f\n", x[1].var, fteval(system, params, dparams_x1))
@printf("%20s %.5f\n", x[2].var, fteval(system, params, dparams_x2))
@printf("%20s %.5f\n", x[3].var, fteval(system, params, dparams_x3))
@printf("%20s %.5f\n", y[1].var, fteval(system, params, dparams_y1))
@printf("%20s %.5f\n", y[2].var, fteval(system, params, dparams_y2))
@printf("%20s %.5f\n", z.var, fteval(system, params, dparams_z))

### Example 3

This example is to present **a wrong case** of Example 2.
Generating the redundancy of ftevent should use different objects.
Please do not mistake it.

#### Definition of component failure events

In [ ]:
x = ftevent("motor failure")
y = ftevent("sensor failure")
z = ftevent("battery failure")

#### Definition of fault tree to represent the system failure

In [ ]:
sub = x + z
system = ftkofn(2, sub, sub, sub) + (y * y) ## sub and y are not redundant!

#### Drawing a brief fault tree.

In [ ]:
draw(todot(system))

In [ ]:
top, forest = bdd(system)
draw(todot(forest, top))

#### Minimal cut set

In [ ]:
ftmcs(system)

#### Define failure probabilities for compoments

In [ ]:
params = Dict(
    x.var => 0.001,
    y.var => 0.0001,
    z.var => 0.00001
)

#### Compute the system failure probability

In [ ]:
fteval(system, params)

#### Birunbaum importance measure

In [ ]:
# define the derivative for a parameter.
dparams_x = Dict(x.var => 1.0, y.var => 0.0, z.var => 0.0)
dparams_y = Dict(x.var => 0.0, y.var => 1.0, z.var => 0.0)
dparams_z = Dict(x.var => 0.0, y.var => 0.0, z.var => 1.0)

In [ ]:
# importance for x
using Printf
@printf("%20s %.5f\n", x.var, fteval(system, params, dparams_x))
@printf("%20s %.5f\n", y.var, fteval(system, params, dparams_y))
@printf("%20s %.5f\n", z.var, fteval(system, params, dparams_z))